## Scratch Book to work out overview image

In [1]:
### COPY OF DEPENDENCIES AND SET UP FROM ADAPTABLE CODE BOOK

# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase

import os


import seaborn as sns
import sqlite3

### Set the folder paths for the data
### DATA FOLDER PATHS
data_folder = '../data/' # ROSTER DATA & GAME STATS
## IMAGES FOLDER PATHS
logo_folder = '../images/logos/' 
background_folder = '../images/backgrounds/'

# Roster Data Paths
roster_path_current = 'rosters/2023_master_roster.csv'
roster_path_previous = 'rosters/2022_master_roster.csv'
# Once the Database Clean and Transform code is updated to insert the roster, we can use the data from the database

## Load rosters into dataframes
df_recent = pd.read_csv(data_folder + roster_path_current)
df_last = pd.read_csv(data_folder + roster_path_previous)


## HOTFIX#####
# Replace dashes with spaces in the School column of the roster dataframes
df_recent['School'] = df_recent['School'].str.replace('-', ' ')
df_last['School'] = df_last['School'].str.replace('-', ' ')


In [2]:
# TEST PARAMETERES

## Set paramaeters

### Set Matchup Tag - This will be used as a folder name to save the charts
matchup_tag = 'MSU_OSU_NEW' #

## Teams to Highlight - Use dashes instead of spaces
team_1 = 'Michigan State' # 
team_1_abrv = 'MSU' #
team_2 = 'Ohio State' #
team_2_abrv = 'OSU' #

## Path to School Logos
team_1_logo =  'msu.png' #
team_2_logo =  'osu.png' #
team_logos = {team_1: logo_folder + team_1_logo, team_2: logo_folder + team_2_logo}

## Set Colors
team_1_color =  'darkgreen' # For Michigan State
team_2_color =  'darkred' # For Ohio State
team_colors = {team_1: team_1_color, team_2: team_2_color}
############ Eventaully want to create a dictionary of colors for each team and save in a file to import

team_abrv_mapping = {
    'Michigan State': 'MSU',
    'Ohio State': 'OSU'
}


### Create a disctionary called highlight_colors to store the colors for each team
### Do it this way so i can reuse code from MSU_Explore_and_Plot notebook
highlight_colors = {team_1_abrv: team_1_color, team_2_abrv: team_2_color}

#### Set Background Image
path_to_bg_image =  'msu_osu_2.png' #
alpha_value = 0.1 # Set the transparency of the background image
## Load the background image
bg_img = mpimg.imread(background_folder + path_to_bg_image)



# Adjust the default chart size and aspect ratio
### SET THE FIGURE SIZE OF THE CHARTS ###
fig_deafult_1 = (10, 8) # Original
fig_deafult_2 = (12, 10)
# Turn on/off the figure size by commenting/uncommenting the lines below
figure_size = fig_deafult_1
# figure_size = fig_deafult_2


################################
## FONT PARAMETERES
font_title_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 36}
font_label_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 16}
font_tick_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 20}
font_label_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 20}
font_tick_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 16}
#################################





In [3]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl


## Filter Dataframe to only the team or teams we are looking at

In [4]:


## Function to compare the members of each team to the previous year and return information about roster turnover

from collections import Counter

def compare_rosters(df_recent, df_last, team):
    # Filter the dataframes by the team
    df_recent_team = df_recent[df_recent['School'] == team]
    df_last_team = df_last[df_last['School'] == team]
    
    # Set of player names for easier comparison
    recent_players = set(df_recent_team['Name'])
    last_players = set(df_last_team['Name'])
    
    # Calculate turnover stats
    new_players = recent_players - last_players
    departed_players = last_players - recent_players
    retained_players = recent_players.intersection(last_players)
    
    # Additional stats like turnover rate, etc.
    turnover_rate = len(departed_players) / len(last_players) * 100 if len(last_players) > 0 else 0
    
    # Count of players in each class rank for both data sets
    recent_class_count = Counter(df_recent_team['Yr.'])
    last_class_count = Counter(df_last_team['Yr.'])
    
    # Calculate how many players from each class were departed and retained
    departed_class_count = Counter(df_last_team[df_last_team['Name'].isin(departed_players)]['Yr.'])
    retained_class_count = Counter(df_last_team[df_last_team['Name'].isin(retained_players)]['Yr.'])
    
    return {
        'new_players': len(new_players),
        'departed_players': len(departed_players),
        'retained_players': len(retained_players),
        'turnover_rate': turnover_rate,
        'recent_class_count': recent_class_count,
        'last_class_count': last_class_count,
        'departed_class_count': departed_class_count,
        'retained_class_count': retained_class_count
    }


# Define a list of teams for analysis (you can replace this list with 'all_teams' or any specific list you have)
# teams_to_analyze = [team_1, team_2]

# Get unique team names from the roster data
all_teams = df_recent['School'].unique()
# Use the entire list of teams
teams_to_analyze = all_teams

# Initialize an empty list to store results
results_list = []

# Loop through each team and get the turnover stats
for team in teams_to_analyze:
    team_stats = compare_rosters(df_recent, df_last, team)
    team_stats['team'] = team  # Add team name to the result dictionary
    results_list.append(team_stats)

# Convert the list of dictionaries to a DataFrame
df_turnover_stats = pd.DataFrame(results_list)





df_turnover_stats.head()
# df_recent.head()

,new_players,departed_players,retained_players,turnover_rate,recent_class_count,last_class_count,departed_class_count,retained_class_count,team
0,13,11,23,32.352941,"{'Fr': 10, 'Gr': 5, 'Sr': 8, 'So': 11, 'Jr': 2}","{'Sr': 6, 'Jr': 9, 'So': 6, 'Fr': 11, 'Gr': 2}","{'So': 4, 'Gr': 2, 'Sr': 3, 'Jr': 2}","{'Sr': 3, 'Jr': 7, 'Fr': 11, 'So': 2}",American Intl
1,13,11,17,39.285714,"{'Fr': 12, 'Sr': 8, 'Jr': 8, 'So': 2}","{'Jr': 10, 'Sr': 8, 'So': 8, 'Fr': 2}","{'Jr': 2, 'Sr': 8, 'So': 1}","{'Jr': 8, 'So': 7, 'Fr': 2}",Lake Superior
2,11,9,20,31.034483,"{'Fr': 11, 'So': 9, 'Jr': 7, 'Sr': 4}","{'Fr': 10, 'Sr': 8, 'So': 7, 'Jr': 4}","{'Sr': 8, 'Fr': 1}","{'Fr': 9, 'So': 7, 'Jr': 4}",Army
3,12,13,15,46.428571,"{'Fr': 8, 'So': 6, 'Jr': 8, 'Sr': 3, 'Gr': 2}","{'Fr': 6, 'Sr': 9, 'Jr': 4, 'So': 8, 'Gr': 1}","{'Sr': 9, 'Jr': 2, 'Gr': 1, 'So': 1}","{'Fr': 6, 'So': 7, 'Jr': 2}",Bowling Green
4,14,12,16,42.857143,"{'Fr': 10, 'So': 8, 'Gr': 3, 'Jr': 6, 'Sr': 3}","{'Fr': 8, 'Gr': 5, 'Jr': 3, 'So': 10, 'Sr': 2}","{'Gr': 5, 'Jr': 1, 'Sr': 2, 'So': 4}","{'Fr': 8, 'So': 6, 'Jr': 2}",Vermont


In [5]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,School
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl


In [6]:
## Rename column on df_recent and _df_last to match the column on df_turnover_stats
df_recent = df_recent.rename(columns={'School': 'team'})
df_last = df_last.rename(columns={'School': 'team'})

# Calculate and store the avera age and experience level of each team
## Age - Use the DOB column to calculate each players age in data, average the age of each team, convert back to years (3 decimal places)
df_recent['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_recent['DOB'])).dt.days
df_recent['Age'] = df_recent['Age'] / 365


df_last['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_last['DOB'])).dt.days
df_last['Age'] = df_last['Age'] / 365


## Experience - Use the Yr. column to calculate each players experience in years, average the experience of each team, convert back to years (3 decimal places)
class_rank_mapping = {'Fr': 0, 'So': 1, 'Jr': 2, 'Sr': 3, 'Gr': 4}
df_recent['Experience'] = df_recent['Yr.'].map(class_rank_mapping)
df_last['Experience'] = df_last['Yr.'].map(class_rank_mapping)

# Caluclate the average age and experience of each team in each dataframe and store in a dictionary
team_age_dict_current = df_recent.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_current = df_recent.groupby('team')['Experience'].mean().round(3).to_dict()
team_age_dict_last = df_last.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_last = df_last.groupby('team')['Experience'].mean().round(3).to_dict()

## Add the average age and experience df_turnover_stats - Use the dictionaries created above, add current as age_current, add last as age_last
df_turnover_stats['age_current'] = df_turnover_stats['team'].map(team_age_dict_current)
df_turnover_stats['age_last'] = df_turnover_stats['team'].map(team_age_dict_last)

df_turnover_stats['exp_current'] = df_turnover_stats['team'].map(team_exp_dict_current)
df_turnover_stats['exp_last'] = df_turnover_stats['team'].map(team_exp_dict_last)


df_turnover_stats.head()

,new_players,departed_players,retained_players,turnover_rate,recent_class_count,last_class_count,departed_class_count,retained_class_count,team,age_current,age_last,exp_current,exp_last
0,13,11,23,32.352941,"{'Fr': 10, 'Gr': 5, 'Sr': 8, 'So': 11, 'Jr': 2}","{'Sr': 6, 'Jr': 9, 'So': 6, 'Fr': 11, 'Gr': 2}","{'So': 4, 'Gr': 2, 'Sr': 3, 'Jr': 2}","{'Sr': 3, 'Jr': 7, 'Fr': 11, 'So': 2}",American Intl,22.652,23.575,1.639,1.471
1,13,11,17,39.285714,"{'Fr': 12, 'Sr': 8, 'Jr': 8, 'So': 2}","{'Jr': 10, 'Sr': 8, 'So': 8, 'Fr': 2}","{'Jr': 2, 'Sr': 8, 'So': 1}","{'Jr': 8, 'So': 7, 'Fr': 2}",Lake Superior,22.351,23.969,1.400,1.857
2,11,9,20,31.034483,"{'Fr': 11, 'So': 9, 'Jr': 7, 'Sr': 4}","{'Fr': 10, 'Sr': 8, 'So': 7, 'Jr': 4}","{'Sr': 8, 'Fr': 1}","{'Fr': 9, 'So': 7, 'Jr': 4}",Army,22.459,23.561,1.129,1.345
3,12,13,15,46.428571,"{'Fr': 8, 'So': 6, 'Jr': 8, 'Sr': 3, 'Gr': 2}","{'Fr': 6, 'Sr': 9, 'Jr': 4, 'So': 8, 'Gr': 1}","{'Sr': 9, 'Jr': 2, 'Gr': 1, 'So': 1}","{'Fr': 6, 'So': 7, 'Jr': 2}",Bowling Green,22.080,23.477,1.444,1.679
4,14,12,16,42.857143,"{'Fr': 10, 'So': 8, 'Gr': 3, 'Jr': 6, 'Sr': 3}","{'Fr': 8, 'Gr': 5, 'Jr': 3, 'So': 10, 'Sr': 2}","{'Gr': 5, 'Jr': 1, 'Sr': 2, 'So': 4}","{'Fr': 8, 'So': 6, 'Jr': 2}",Vermont,21.910,23.043,1.367,1.500


## Add Draft Information to DF

In [7]:
df_recent.head()

,Unnamed: 0,No.,Name,Yr.,Ht.,Wt.,DOB,Hometown,Position,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,team,Age,Experience
0,NaN,23,"Angesved, Theo",Fr,5-10,172,6/23/2003,"Växjö, Sweden",Defensemen,70,NaN,NaN,NaN,Frölunda HC J20,SWE,American Intl,20.367123,0
1,NaN,4,"Callahan, Brett",Gr,5-10,182,10/16/1998,"Westmont, Ill.",Defensemen,70,NaN,NaN,NaN,Jersey Hitmen,USPHL,American Intl,25.054795,4
2,NaN,7,"Kramer, Brian",Sr,5-11,181,7/20/2000,"Wexford, Pa.",Defensemen,71,NaN,NaN,NaN,Fargo,USHL,American Intl,23.293151,3
3,NaN,5,"Leppanen, Tomi",Fr,6-1,190,4/7/2003,"Heinola, Finland",Defensemen,73,NaN,NaN,NaN,Sioux City,USHL,American Intl,20.578082,0
4,NaN,20,"McCurdy, Hunter",So,6-1,220,8/9/2001,"Mokena, Ill.",Defensemen,73,NaN,NaN,NaN,Danbury,NAHL,American Intl,22.238356,1


In [8]:
# Create new column called "Drafted" and assign a value of 1 if the player was drafted, 0 if not
df_recent['Drafted'] = np.where(df_recent['Draft_Year'] > 0, 1, 0)
df_last['Drafted'] = np.where(df_last['Draft_Year'] > 0, 1, 0)



# Calculate the Draft Score Based on round drafted
### NEW DRAFT SCORE CALCULATION ###
# Assign weights to draft rounds
draft_weights = {
    1: 8,  # Extra weight for 1st round
    2: 6,
    3: 5,
    4: 4,
    5: 3,
    6: 2,
    7: 1
}

# Compute player draft scores and fill NaNs with 0
df_recent['Player_Draft_Score'] = df_recent['D_Round'].map(draft_weights).fillna(0)
df_last['Player_Draft_Score'] = df_last['D_Round'].map(draft_weights).fillna(0)

## Create new dataframes with team, count of drafted players, and sum of draft scores
df_recent_count = df_recent.groupby('team')['Drafted'].sum().reset_index()
df_recent_sum = df_recent.groupby('team')['Player_Draft_Score'].sum().reset_index()

df_last_count = df_last.groupby('team')['Drafted'].sum().reset_index()
df_last_sum = df_last.groupby('team')['Player_Draft_Score'].sum().reset_index()


# Merge the count and sum dataframes into single dataframe
df_recent_draft_score = df_recent_count.merge(df_recent_sum, on='team', how='left')
df_last_draft_score = df_last_count.merge(df_last_sum, on='team', how='left')

# Rename the columns
df_recent_draft_score = df_recent_draft_score.rename(columns={'Drafted': 'Drafted_Count', 'Player_Draft_Score': 'Player_Draft_Score_Sum'})
df_last_draft_score = df_last_draft_score.rename(columns={'Drafted': 'Drafted_Count_Previous', 'Player_Draft_Score': 'Player_Draft_Score_Sum_Previous'})

# Merge the two dataframes into one
df_draft_score = df_recent_draft_score.merge(df_last_draft_score, on='team', how='left')

# Normalize the two columns of draft scores to a 0-100 scale
df_draft_score['Normalized_Draft_Score'] = df_draft_score['Player_Draft_Score_Sum'] / df_draft_score['Player_Draft_Score_Sum'].max() * 100
df_draft_score['Normalized_Draft_Score_Previous'] = df_draft_score['Player_Draft_Score_Sum_Previous'] / df_draft_score['Player_Draft_Score_Sum_Previous'].max() * 100

# Calculate the change in draft score from last year
df_draft_score['Draft_Score_Change'] = df_draft_score['Normalized_Draft_Score'] - df_draft_score['Normalized_Draft_Score_Previous']
## Switch and store as percentage change
df_draft_score['Draft_Score_Change'] = df_draft_score['Draft_Score_Change'] / df_draft_score['Normalized_Draft_Score_Previous'] * 100

# Merge into the df_turnover_stats dataframe
df_turnover_stats = df_turnover_stats.merge(df_draft_score[['team', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']], on='team', how='left')

# Reorder columns so team is first
df_turnover_stats = df_turnover_stats[['team', 'new_players', 'departed_players', 'retained_players', 'turnover_rate', 'recent_class_count', 'last_class_count', 'departed_class_count', 'retained_class_count', 'age_current', 'age_last', 'exp_current', 'exp_last', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']]

# Calculate and store ranking for the Age and Experience columns
df_turnover_stats['age_rank_current'] = df_turnover_stats['age_current'].rank(ascending=False) # Oldest = 1 
df_turnover_stats['exp_rank_current'] = df_turnover_stats['exp_current'].rank(ascending=False) # Oldest = 1


## Save dataframe as csv in TEMP folder
df_turnover_stats.to_csv('../TEMP/df_turnover_stats.csv', index=False)

df_turnover_stats.head()



,team,new_players,departed_players,retained_players,turnover_rate,recent_class_count,last_class_count,departed_class_count,retained_class_count,age_current,age_last,exp_current,exp_last,Drafted_Count,Drafted_Count_Previous,Draft_Score_Change,Normalized_Draft_Score,Normalized_Draft_Score_Previous,age_rank_current,exp_rank_current
0,American Intl,13,11,23,32.352941,"{'Fr': 10, 'Gr': 5, 'Sr': 8, 'So': 11, 'Jr': 2}","{'Sr': 6, 'Jr': 9, 'So': 6, 'Fr': 11, 'Gr': 2}","{'So': 4, 'Gr': 2, 'Sr': 3, 'Jr': 2}","{'Sr': 3, 'Jr': 7, 'Fr': 11, 'So': 2}",22.652,23.575,1.639,1.471,0,0.0,NaN,0.000000,0.000000,11.0,26.0
1,Lake Superior,13,11,17,39.285714,"{'Fr': 12, 'Sr': 8, 'Jr': 8, 'So': 2}","{'Jr': 10, 'Sr': 8, 'So': 8, 'Fr': 2}","{'Jr': 2, 'Sr': 8, 'So': 1}","{'Jr': 8, 'So': 7, 'Fr': 2}",22.351,23.969,1.400,1.857,0,1.0,-100.000000,0.000000,1.351351,25.0,43.0
2,Army,11,9,20,31.034483,"{'Fr': 11, 'So': 9, 'Jr': 7, 'Sr': 4}","{'Fr': 10, 'Sr': 8, 'So': 7, 'Jr': 4}","{'Sr': 8, 'Fr': 1}","{'Fr': 9, 'So': 7, 'Jr': 4}",22.459,23.561,1.129,1.345,0,0.0,NaN,0.000000,0.000000,20.0,58.0
3,Bowling Green,12,13,15,46.428571,"{'Fr': 8, 'So': 6, 'Jr': 8, 'Sr': 3, 'Gr': 2}","{'Fr': 6, 'Sr': 9, 'Jr': 4, 'So': 8, 'Gr': 1}","{'Sr': 9, 'Jr': 2, 'Gr': 1, 'So': 1}","{'Fr': 6, 'So': 7, 'Jr': 2}",22.080,23.477,1.444,1.679,1,0.0,inf,4.761905,0.000000,33.0,41.5
4,Vermont,14,12,16,42.857143,"{'Fr': 10, 'So': 8, 'Gr': 3, 'Jr': 6, 'Sr': 3}","{'Fr': 8, 'Gr': 5, 'Jr': 3, 'So': 10, 'Sr': 2}","{'Gr': 5, 'Jr': 1, 'Sr': 2, 'So': 4}","{'Fr': 8, 'So': 6, 'Jr': 2}",21.910,23.043,1.367,1.500,4,5.0,8.424908,19.047619,17.567568,41.0,46.5
